# QAMP-15: Building out Qiskit-QEC with the XP Stabilizer Formalism

Members: Dhruv Bhatnagar, __Ruihao Li__

Mentors: Grace Harper, Drew Vandeth (IBM)

Implementation based on the following work: <br>
- M. A. Webster, B. J. Brown, and S. D. Bartlett, The XP Stabiliser Formalism: a Generalisation of the Pauli Stabiliser Formalism with Arbitrary Phases. *Quantum* __6__, 815 (2022).
- Mark Webster's [XPFpackage](https://github.com/m-webster/XPFpackage)

## Setting things up

In [1]:
!git clone --branch qamp-qiskit-demodays https://github.com/qiskit-community/qiskit-qec.git /Users/ruihaoli/qiskit-qec

fatal: destination path '/Users/ruihaoli/qiskit-qec' already exists and is not an empty directory.


In [2]:
%cd ../qiskit-qec

/Users/ruihaoli/qiskit-qec


In [3]:
!pip install -r requirements.txt

This talk will consist of two parts:

- XP operator algebra
- Linear algebra modulo $N$ and the Howell matrix form

## XP Operator Algebra


The XP operator algebra is a fundamental building block of the XP stabilizer formalism. Here we show how to express algebraic operations in terms of the vector representation and the generalized symplectic product. These concepts allow us to determine the eigenvalues and actions of the projectors of XP operators.

In the Pauli stabilizer formalism, one represents operators with the _symplectic representation_, i.e., with binary/boolean vectors $(\mathbf{x}, \mathbf{z})$ and a boolean phase $p$. Such representation is naturally generalized in the XP formalism.

Let $\mathbf{u} = (p | \mathbf{x} | \mathbf{z}) \in \mathbb Z\times \mathbb Z^n\times \mathbb Z^n$ be an integer vector of length $2n+1$. Recall that the XP operator of precision $N$ corresponding to $\mathbf{u}$ is defined as
$$
XP_N(\mathbf{u}) = \omega^p \bigotimes_{0\leq i < n} X^{\mathbf{x}_i} P^{\mathbf{z}_i},
$$
where
$$
\begin{split}
\omega &= \exp\left(\frac{\pi i}{N}\right), \\
P &= \text{diag}(1, \omega^2). \\
\end{split}
$$
Note that each component is periodic such that
$$
XP_N(p | \mathbf{x} | \mathbf{z}) = XP_N(p\ \text{mod}\ 2N | \mathbf{x}\ \text{mod}\ 2 | \mathbf{z}\ \text{mod}\ N).
$$
Therfore, we can define a __unique vector representation__ $(p | \mathbf{x} | \mathbf{z}) \in \mathbb Z_{2N}\times \mathbb Z^n_{2}\times \mathbb Z^n_N$ for each XP operator.

In [4]:
import numpy as np
from qiskit_qec.operators.xp_pauli import XPPauli
from qiskit_qec.operators.xp_pauli_list import XPPauliList

In [5]:
# XPPauli object
a = XPPauli(
    data=np.array([0, 3, 1, 6, 4, 3], dtype=np.int64), phase_exp=11, precision=4
)
unique_a = a.unique_vector_rep()
print(unique_a.matrix)
print(unique_a.x)
print(unique_a.z)
print(unique_a._phase_exp)

[[0 1 1 2 0 3]]
[0 1 1]
[2 0 3]
[3]


In [6]:
# XPPauliList object
b = XPPauliList(
    data=np.array([[1, 0, 1, 1, 5, 3, 5, 4], [1, 0, 1, 0, 1, 5, 2, 0]], dtype=np.int64),
    phase_exp=np.array([4, 7]),
    precision=6,
)
unique_b = b.unique_vector_rep()
print(unique_b.matrix)
print(unique_b._phase_exp)

[[1 0 1 1 5 3 5 4]
 [1 0 1 0 1 5 2 0]]
[4 7]


When possible, we can also rescale the vector representation of an XP operator to one with a different precision $N'$. For example, for $XP_8(12|1110000|0040000)$, since the phase and $Z$ components are divisible by 4, we can rescale it to a precision-2 operator:
$$
XP_8(12|1110000|0040000) = XP_2(3|1110000|0010000).
$$

In [7]:
a = XPPauli(
    data=np.array([1, 1, 1, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0]), phase_exp=12, precision=8
)
rescaled_a = a.rescale_precision(new_precision=2, inplace=False)
print(rescaled_a.matrix)
print(rescaled_a._phase_exp)

[[1 1 1 0 0 0 0 0 0 1 0 0 0 0]]
[3]


In [8]:
# Case where it is not possible to rescale
a.rescale_precision(new_precision=3, inplace=False)

QiskitError: 'XP Operator cannot be expressed in new_precision.'

Many of the algebraic operations within the XP formalism relies on the __antisymmetric operator__ of precision $N$ corresponding to a vector $\mathbf{z}\in \mathbb Z^n$:
$$
D_{N}(\mathbf{z}) = XP_N(\sum_i \mathbf{z}_i | \mathbf{0} | -\mathbf{z}).
$$

In [9]:
a = XPPauli(data=np.array([1, 1, 2, 2, 1, 2, 3, 3]), phase_exp=0, precision=8)
antisym_op = a.antisymmetric_op(int_vec=a.z)
print(antisym_op.matrix)
print(antisym_op._phase_exp)

[[ 0  0  0  0 -1 -2 -3 -3]]
[9]


For example, the product of two XP operators in vector form is given by
$$
XP_N(\mathbf{u}_1) XP_N(\mathbf{u}_2) = XP_N(\mathbf{u}_1 + \mathbf{u}_2) D_N(2\mathbf{u}_1 \mathbf{u}_2),
$$
where in this notation the arithmetic operations on vectors are component-wise in $\mathbb Z$, i.e.,
$$
\begin{split}
(\mathbf a + \mathbf b)[i] &= \mathbf a[i] + \mathbf b[i], \\
(\mathbf a \mathbf b)[i] &= \mathbf a[i] \mathbf b[i].
\end{split}
$$

In Qiskit-QEC, the multiplication operation is called `compose`.

In [10]:
a = XPPauli(data=np.array([0, 1, 0, 0, 2, 0]), phase_exp=6, precision=4)
b = XPPauli(data=np.array([1, 1, 1, 3, 3, 0]), phase_exp=2, precision=4)
product = a.compose(b, inplace=False)
print(product.matrix)
print(product._phase_exp)

[[1 0 1 3 3 0]]
[6]


In [11]:
# Multiplying two XPPauliList objects
a = XPPauliList(
    data=np.array([[1, 0, 1, 1, 5, 3, 5, 4], [1, 0, 1, 0, 1, 5, 2, 0]]),
    phase_exp=np.array([4, 7]),
    precision=6,
)
b = XPPauliList(
    data=np.array([[1, 0, 0, 1, 4, 1, 0, 1], [0, 1, 1, 0, 1, 3, 0, 5]]),
    phase_exp=np.array([11, 2]),
    precision=6,
)
product = a.compose(b, inplace=False)
print(product.matrix)
print(product._phase_exp)

[[0 0 1 0 1 4 5 3]
 [1 1 0 0 0 2 2 5]]
[ 1 11]


The closed form expressions for various other algebraic operations are given in Table 4 of the original paper, which is attached below:

<div align="center">
<img src="img/xp_algebra_tab.png" width="800"/>
</div>

Below is the correspondence between these operations and the Qiskit-QEC methods:
- MUL: `compose`
- POW: `power` (note that SQ is a special case of POW)
- INV: `inverse`
- CONJ: `conjugate`
- COMM: `commutator`

In [12]:
a = XPPauliList(
    data=np.array([[1, 0, 1, 1, 5, 3, 5, 4], [1, 0, 1, 1, 5, 4, 1, 5]]),
    phase_exp=[1, 2],
    precision=6,
)
a1 = a.power(n=5)
print(a1.matrix)
print(a1._phase_exp, "\n")

a2 = a.power(n=[3, 4])
print(a2.matrix)
print(a2._phase_exp)

[[1 0 1 1 5 3 5 4]
 [1 0 1 1 5 2 1 5]]
[1 6] 

[[1 0 1 1 5 3 5 4]
 [0 0 0 0 0 4 0 0]]
[7 4]


There are also other methods we implemented, such as `weight`, `degree`, `fundamental_phase`, etc., which would be useful for identifying the codespace and measurements in the XP formalism (not implemented). I won't go into details here.

## Linear Algebra Modulo $N$

As can be seen from the periodic nature of the XP operators, we are working with vectors over $\mathbb Z_N$, which is a __ring__ in general rather than a field. To find codewords and logical operators etc. in the XP formalism, we need to apply linear algebra techniques over rings, i.e., modulo $N$. So we first need to implement basic arithmetics over $\mathbb Z_N$.

A full list of the arithmetic operations implemented includes:
- `quo`: Quotient of $a/b$ in $\mathbb Z_N$.
- `div`: Divisor of $a$ & $b$ in $\mathbb Z_N$ - $d\in \mathbb Z_N$ such that $(b  d)\ \text{mod}\ N  = a\ \text{mod}\ N$.
- `ann`: Annihilator of $a$ in $\mathbb Z_N$ - $b\in \mathbb Z_N$ such that $(a b)\ \text{mod}\ N  = 0$.
- `stab`: Stabilizer of $a$ & $b$ in $\mathbb Z_N$ - $c\in \mathbb Z_N$ such that $\text{GCD}_{\mathbb Z_N}(a+b c, N) = \text{GCD}_{\mathbb Z_N}(a, b, N)$.
- `unit`: Unit of $a$ in $\mathbb Z_N$ - $u\in \mathbb Z_N$ such that $(a u)\ \text{mod}\ N  = \text{GCD}_{\mathbb Z_N}(a, N)$.
- `gcd_ext`: Extended Euclidean algorithm for finding the GCD of $a$ & $b$ in $\mathbb Z_N$ and the corresponding coefficients (explained in detail below).

In [13]:
import qiskit_qec.arithmetic.modn as modn

In [14]:
# Quotient of a/b in the ring Z_N
a, b, N = 18, 3, 5
q = modn.quo(a, b, N)
print(q)
print(q == (a % N) // (b % N))

1
True


In [15]:
# Divisor of a/b in the ring Z_N
a, b, N = 24, 8, 7
d = modn.div(a, b, N)
print(d)
print((b * d) % N == a % N)

3
True


In [16]:
# Annihilator of a in the ring Z/nZ
a, N = 10, 5
ann = modn.ann(a, N)
print(ann)
print((a * ann) % N == 0)

1
True


An important arithmetic operation is the extended Euclidean algorithm for finding the __greatest common divisor__ (GCD) of two integers $a$ and $b$ in $\mathbb Z_N$. For any $a, b\in \mathbb Z_N$, the `gcd_ext` function outputs the tuple $(g, s, t, u, v)$ which satisfies the following properties:
- $g = \text{GCD}_{\mathbb Z_N}(a, b, N) = (sa + tb)\ \text{mod}\ N$
- $(ua + vb)\ \text{mod}\ N = 0$
- $(sv - tu)\ \text{mod}\ N = 1$

Note that $s$ & $t$ are sometimes called the Bézout coefficients.

Next, we are ready to implement the algorithm for calculating the __Howell form__ of a matrix $A$ over $\mathbb Z_N$. The Howell matrix form is a generalization of the Reduced Row Echelon Form (RREF) which gives us a canonical basis for the row span of a matrix over a ring. This is needed for determining the codespaces in the XP formalism.

Below is the algorithm for calculating the Howell matrix form:

1. Set the row index $r=0$ and column index $c=0$
2. Determine if there exists some $A[j,c]> 0$ for $j \ge r$. If such a $j$ exists:
    - Ensure that $A[r,c] > 0$ by performing the SWAP row operation $\begin{pmatrix}A[r]\\A[j]\end{pmatrix} := \begin{pmatrix}0&1\\1&0\end{pmatrix}\begin{pmatrix}A[r]\\A[j]\end{pmatrix}$ if necessary 
    - Eliminate any entries below $A[r,c]$ by doing the following. For each $j > r$ with $A[j,c] > 0$, let $(g,s,t,u,v) = \text{Gcdex}(A[r,c],A[j,c],N)$. Perform the row operation $\begin{pmatrix}A[r]\\A[j]\end{pmatrix} := \begin{pmatrix}s&t\\u&v\end{pmatrix}\begin{pmatrix}A[r]\\A[j]\end{pmatrix}$. This ensures that $A[r,c] = g$ where $g = \text{GCD}_{\mathbb{Z}_N}(A[r,c],A[j,c])$ and $A[j,c] = 0$. This operation is span-preserving because $sv-tu = 1$.
    - Ensure that $A[r,c]$ is a minimal representative by finding a unit $u$ such that $uA[r,c]= \text{GCD}_{\mathbb{Z}_N}(A[r,c],N)$. Perform the row operation $A[r] := u A[r]$.
    - Ensure that $A[j,c] < A[r,c]$ for $j < r$ by determining $s = A[j,c] // A[r,c]$ (ie integer division). Perform the row operation $\begin{pmatrix}A[r]\\A[j]\end{pmatrix} := \begin{pmatrix}1&0\\-s&1\end{pmatrix}\begin{pmatrix}A[r]\\A[j]\end{pmatrix}$.
    - If $A[r,c]$ is a zero divisor, find the annihilator $a$ such that $a A[r,c]= 0$ and perform the row operation $A.\text{append}(aA[r])$ to append a new row. This ensures that the Howell property holds
    - Increment $r$ - halt if the number of rows in $A$ is equal to $r$
3. Increment $c$ - halt if the number of columns in $A$ is equal to $c$

Essentially, the above algorithm relies on a set of span-preserving row operations, which we have implemented within the `do_row_op` function in `qiskit_qec.linear.matrix`. The row operation options include `'swap'`, `'unit'`, `'add'`, `'append'`, and `'update'`.

In [17]:
from qiskit_qec.linear.matrix import do_row_op

mat = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
N = 4
print(mat, "\n")

# Swap rows 0 and 1
mat1 = do_row_op(mat, ("swap", [0, 1], []), N)
print(mat1, "\n")

# Append the product of row 0 by a scalar (3) to the end of matrix
mat2 = do_row_op(mat, ("append", [0], [3]), N)
print(mat2)

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[4 5 6]
 [1 2 3]
 [7 8 9]] 

[[1 2 3]
 [4 5 6]
 [7 8 9]
 [3 2 1]]


The Howell transformation comes in two functions: 
- `howell`: outputs just the Howell matrix $H$ for a given matrix $A$
- `howell_complete`, outputs the Howell matrix $H$, the corresponding transformation matrix $U$ s.t. $UA = H$, and the kernel matrix $K$ s.t. $KA = 0$.

This is similar to the design of RREF in Qiskit-QEC (`rref` & `rref_complete`).

In [18]:
from qiskit_qec.linear.matrix import howell, howell_complete

mat = np.array([[8, 5, 5], [0, 9, 8], [0, 0, 10]])
N = 12
howell_mat = howell(mat, N)
print(howell_mat, "\n")

howell_mat, transform_mat, kernel_mat = howell_complete(mat, N)
print(howell_mat, "\n")
print(transform_mat, "\n")
print(kernel_mat)

[[4 1 0]
 [0 3 0]
 [0 0 1]] 

[[4 1 0]
 [0 3 0]
 [0 0 1]] 

[[8 1 0]
 [0 7 4]
 [9 3 4]] 

[[6 6 3]
 [0 4 4]]


## Final Remarks

The implementations introduced above are included in PR [#281](https://github.com/qiskit-community/qiskit-qec/pull/281), [#304](https://github.com/qiskit-community/qiskit-qec/pull/304), and [#279](https://github.com/qiskit-community/qiskit-qec/pull/279).

<div align="center">
<img src="img/pr_281.png" width="800"/>
</div>

<div align="center">
<img src="img/pr_304.png" width="800"/>
</div>

<div align="center">
<img src="img/pr_279.png" width="800"/>
</div>

I would like to thank Grace Haper and Drew Vandeth from IBM for such a great opportunity and their mentoring and feedbacks. I would also like to thank Dhruv Bhatnagar for the collaboration on this fun project.